In [1]:
import nltk
import string
import os
import random
import sklearn
import numpy as np
import pandas as pd
import math
import re

from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from random import sample

from sklearn.model_selection import StratifiedShuffleSplit


from nltk.util import ngrams
from nltk import word_tokenize

from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer



# Download NLTK resources
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# THIS IS FROM HARISH'S DATASET

from google.colab import drive
drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
# Step 1: Load the dataset

def load_data(folder_path, label):
    # Initialize empty lists to store data and labels
    data = []

    # Loop through each file in the specified folder_path (i.e., pos/neg)
    for filename in os.listdir(folder_path):
        with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
            # Read the content of the file and append it to the data list
            content = file.read()
            data.append((content, label))

    return data

# Load reviews and their corresponding star ratings
positive_data = load_data("/content/drive/My Drive/Bath/data/pos", "positive")
negative_data = load_data("/content/drive/My Drive/Bath/data/neg", "negative")

In [4]:
# Importing the necessary function from scikit-learn for splitting the dataset
from sklearn.model_selection import train_test_split

# Combining positive and negative data into a single list
all_data = positive_data + negative_data

# Extracting the reviews (features) and labels from the combined data
data = [review[0] for review in all_data]
labels = [review[1] for review in all_data]

# Spillting the data into 80-10-10
# Splitting the data into training (80%), temporary (20% remaining for further splitting)
# The random_state parameter ensures reproducibility of the split
train_data, train_temp, train_labels, labels_temp = train_test_split(data, labels, test_size=0.2, random_state=42)

# Further splitting the temporary data into development (10%) and testing (10%) sets
# The random_state parameter ensures reproducibility of the split
dev_data, test_data, dev_labels, test_labels = train_test_split(train_temp, labels_temp, test_size=0.5, random_state=42)


In [5]:
# FEATURE SELECTION + TOKENISE
# Use a combination of stopwords, lemmatization or stemming and TFIDF to select the features you intend to use.
# You are expected to have at least three different sets of features.

# FROM Week2a TermWeighting.ipynb

# Importing necessary libraries and modules for text processing
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
import string
import re

# STEMMING + LOWER CASE + GET RID OF STOP WORDS + GET RID OF PUNCTUATIONS
def process_one(text):
    tokenised_list = []  # Initialize an empty list to store tokenized words for each document
    stoplist = set(stopwords.words('english'))  # Set of English stopwords
    st = LancasterStemmer()  # Lancaster Stemmer for stemming
    for data in text:
        word_list = [st.stem(re.sub(r'[^a-zA-Z0-9]', '', word)) for word in word_tokenize(data.lower()) if word not in stoplist and not re.match(r'[^a-zA-Z0-9]+', word)]
                  # re.sub(r'[^a-zA-Z0-9]', '', word) replace non-alphanumeric character with empty string (i.e., remove punctuations)
                  # re.match(r'[^a-zA-Z0-9]+', word) filters out words that start with non-alphanumeric characters
                  # a tokenized list of words, all converted to lowercase,
                  # if the word is not in the stoplist and not a punctuation mark
        tokenised_list.append(word_list)
    return tokenised_list

# STEMMING + GET RID OF STOP WORDS + GET RID  OF PUNCTUATIONS
def process_two(text):
    tokenised_list = []  # Initialize an empty list to store tokenized words for each document
    stoplist = set(stopwords.words('english'))  # Set of English stopwords
    st = LancasterStemmer()  # Lancaster Stemmer for stemming
    for data in text:
        word_list = [st.stem(re.sub(r'[^a-zA-Z0-9]', '', word)) for word in word_tokenize(data) if word not in stoplist and not re.match(r'[^a-zA-Z0-9]+', word)]
                    # a tokenized list of words,
                    # if the word is not in the stoplist and not a punctuation mark
        tokenised_list.append(word_list)
    return tokenised_list

# LEMMATIZE + LOWER CASE + GET RID OF STOP WORDS
def process_three(text):
    tokenised_list = []  # Initialize an empty list to store tokenized words for each document
    stoplist = set(stopwords.words('english'))  # Set of English stopwords
    lemmatizer = WordNetLemmatizer()  # WordNet Lemmatizer for lemmatization
    for data in text:
        word_list = [lemmatizer.lemmatize(word) for word in word_tokenize(data.lower()) if word not in stoplist]
                    # a tokenized list of words, all converted to lowercase,
                    # if the word is not in the stoplist
        tokenised_list.append(word_list)
    return tokenised_list

# Tokenizing and processing the training data using the FIRST feature selection
feature_selection_one_training = process_one(train_data)
print("Training Data Tokenized (Feature Selection Method 1):", feature_selection_one_training[:20])
feature_selection_one_development = process_one(dev_data)
print("Development Data Tokenized (Feature Selection Method 1):", feature_selection_one_development[:20])
feature_selection_one_testing = process_one(test_data)
print("Testing Data Tokenized (Feature Selection Method 1):", feature_selection_one_testing[:20])



# Tokenizing and processing the training data using the SECOND feature selection
feature_selection_two_training = process_two(train_data)
#print("Training Data Tokenised (Feature Selection Method 2):", feature_selection_two_training[:20])
feature_selection_two_development = process_two(dev_data)
#print("Development Data Tokenised (Feature Selection Method 2):", feature_selection_two_development[:20])
feature_selection_two_testing = process_two(test_data)
#print("Testing Data Tokenised (Feature Selection Method 2):", feature_selection_two_testing[:20])

# Tokenizing and processing the training data using the THIRD feature selection
feature_selection_three_training = process_three(train_data)
#print("Training Data Tokenised (Feature Selection Method 3):", feature_selection_three_training[:20])
feature_selection_three_development = process_three(dev_data)
#print("Development Data Tokenised (Feature Selection Method 3):", feature_selection_three_development[:20])
feature_selection_three_testing = process_three(test_data)
#print("Testing Data Tokenised (Feature Selection Method 3):", feature_selection_three_testing[:20])


Training Data Tokenized (Feature Selection Method 1): [['bottom', 'barrel', 'unimagin', 'pract', 'unwatch', 'remak', 'road', 'warry', 'film', 'follow', 'exact', 'plot', 'filipino', 'film', 'stryker', 'wors', 'far', 'bad', 'act', 'dialog', 'effect', 'dub', 'pac', 'act', 'sequ', 'list', 'goe', 'ita', 'mad', 'lit', 'doz', 'road', 'warry', 'ripoff', 'ear', '80', 'good', 'bad', 'worst', 'far', 'contest', 'mood', 'film', 'complet', 'bleak', 'mis', 'expery', 'sit', 'on', 'bor', 'half', '1', 'on', 'good', 'chas', 'sequ', 'toward', 'begin', 'movy', 'cool', 'shot', 'man', 'hold', 'hand', 'grenad', 'explod', 'everyth', 'els', 'movy', 'sery', 'reek', 'act', 'postnuk', 'fun', 'go', 'track', 'cop', 'endgam', 'fal', 'new', 'york', 'escap', 'bronx', 'instead', 'much', 'enjoy', 'rub'], ['hothead', 'cop', 'accid', 'kil', 'murd', 'suspect', 'cov', 'crim', 'must', 'deal', 'guil', 'conscy', 'tri', 'solv', 'murd', 'cas', 'andrew', 'tierney', 'reunit', 'direct', 'prem', 'film', 'noir', 'effect', 'laur', 'ear

In [6]:
# FEATURE GENERATION USING N-GRAMS
# Have a systematic method of choosing “n” (1, 2, 3.. ). Detail why you limit your n to whatever you choose.

# Importing necessary library for n-grams
from nltk import ngrams

# Function to generate n-grams from a list of tokenized words
def ngramsfunction(folder, n):
    # Initialize an empty list to store generated n-grams
    ngrams_list = []

    # Iterate through document in the input folder
    for new_gram in folder:
        # Generate n-grams from the tokenized words
        n_grams = list(ngrams(new_gram, n))

        # Convert n-grams to a list of strings by joining the elements with a space
        ngrams_training = [' '.join(gram) for gram in n_grams]

        # Append the list of n-grams to the ngrams_list
        ngrams_list.append(ngrams_training)

    return ngrams_list

# Generate ngrams features for each data set
ngrams_one_training = ngramsfunction(feature_selection_one_training, 1)
print("1", ngrams_one_training[:10])
ngrams_one_development = ngramsfunction(feature_selection_one_development, 1)
print("2", ngrams_one_development[:10])
ngrams_one_testing = ngramsfunction(feature_selection_one_testing, 1)
print("3", ngrams_one_testing[:10])

ngrams_two_training = ngramsfunction(feature_selection_two_training, 1)
ngrams_two_development = ngramsfunction(feature_selection_two_development, 1)
ngrams_two_testing = ngramsfunction(feature_selection_two_testing, 1)
print("\n1", ngrams_two_training[:10])

ngrams_three_training = ngramsfunction(feature_selection_three_training, 1)
ngrams_three_development = ngramsfunction(feature_selection_three_development, 1)
ngrams_three_testing = ngramsfunction(feature_selection_three_testing, 1)
print("\n1", ngrams_three_training[:10])


1 [['bottom', 'barrel', 'unimagin', 'pract', 'unwatch', 'remak', 'road', 'warry', 'film', 'follow', 'exact', 'plot', 'filipino', 'film', 'stryker', 'wors', 'far', 'bad', 'act', 'dialog', 'effect', 'dub', 'pac', 'act', 'sequ', 'list', 'goe', 'ita', 'mad', 'lit', 'doz', 'road', 'warry', 'ripoff', 'ear', '80', 'good', 'bad', 'worst', 'far', 'contest', 'mood', 'film', 'complet', 'bleak', 'mis', 'expery', 'sit', 'on', 'bor', 'half', '1', 'on', 'good', 'chas', 'sequ', 'toward', 'begin', 'movy', 'cool', 'shot', 'man', 'hold', 'hand', 'grenad', 'explod', 'everyth', 'els', 'movy', 'sery', 'reek', 'act', 'postnuk', 'fun', 'go', 'track', 'cop', 'endgam', 'fal', 'new', 'york', 'escap', 'bronx', 'instead', 'much', 'enjoy', 'rub'], ['hothead', 'cop', 'accid', 'kil', 'murd', 'suspect', 'cov', 'crim', 'must', 'deal', 'guil', 'conscy', 'tri', 'solv', 'murd', 'cas', 'andrew', 'tierney', 'reunit', 'direct', 'prem', 'film', 'noir', 'effect', 'laur', 'ear', 'collab', 'andrew', 'perfect', 'cast', 'earnest',

In [7]:
# Importing necessary libraries for calculations
from collections import Counter
import math

# Function to calculate Term Frequency (TF) for a document
def calculate_tf(document):
    term_count = Counter(document)
    total_terms = len(document)
    return {term: count / total_terms for term, count in term_count.items()}

# Function to calculate Inverse Document Frequency (IDF) values for a document set
def calculate_idf_values(document_list):
    idf_values = {}

    # Iterate through each document in the document set
    for document in document_list:
        # Iterate through each unique term in the document
        for term in set(document):
            idf_values[term] = idf_values.get(term, 0) + 1

    total_documents = len(document_list)

    # Calculate IDF values using the formula and return as a dictionary
    return {term: math.log(total_documents / (1 + document_count)) for term, document_count in idf_values.items()}

# Function to calculate TF-IDF values for a document using precomputed IDF values
def calculate_tfidf(document, idf_values):
    tf = calculate_tf(document)

    # Calculate TF-IDF values using the precomputed TF and IDF values
    return {term: tf[term] * idf_values[term] for term in tf}

# Function to calculate TF-IDF values for an entire document set using precomputed IDF values
def calculate_tfidf_values(document_list, idf_values):
    return [calculate_tfidf(document, idf_values) for document in document_list]

# Calculate IDF values for the entire document set for training, development, and testing data
idf_one_training = calculate_idf_values(ngrams_one_training)
idf_one_development = calculate_idf_values(ngrams_one_development)
idf_one_testing = calculate_idf_values(ngrams_one_testing)

# Calculate TF-IDF values for each document set using precomputed IDF values
tfidf_one_training = calculate_tfidf_values(ngrams_one_training, idf_one_training)
tfidf_one_development = calculate_tfidf_values(ngrams_one_development, idf_one_development)
tfidf_one_testing = calculate_tfidf_values(ngrams_one_testing, idf_one_testing)

print(tfidf_one_training[:100])

# Calculate IDF values for the entire document set for training, development, and testing data
idf_two_training = calculate_idf_values(ngrams_two_training)
idf_two_development = calculate_idf_values(ngrams_two_development)
idf_two_testing = calculate_idf_values(ngrams_two_testing)
# Calculate TF-IDF values for each document set using precomputed IDF values
tfidf_two_training = calculate_tfidf_values(ngrams_two_training, idf_two_training)
tfidf_two_development = calculate_tfidf_values(ngrams_two_development, idf_two_development)
tfidf_two_testing = calculate_tfidf_values(ngrams_two_testing, idf_two_testing)
print(tfidf_two_training[:100])

# Calculate IDF values for the entire document set for training, development, and testing data
idf_three_training = calculate_idf_values(ngrams_three_training)
idf_three_development = calculate_idf_values(ngrams_three_development)
idf_three_testing = calculate_idf_values(ngrams_three_testing)
# Calculate TF-IDF values for each document set using precomputed IDF values
tfidf_three_training = calculate_tfidf_values(ngrams_three_training, idf_three_training)
tfidf_three_development = calculate_tfidf_values(ngrams_three_development, idf_three_development)
tfidf_three_testing = calculate_tfidf_values(ngrams_three_testing, idf_three_testing)
print(tfidf_three_training[:100])

[{'bottom': 0.04478757263094461, 'barrel': 0.06420689010344618, 'unimagin': 0.06886740858744807, 'pract': 0.043772715077546, 'unwatch': 0.06328685898076186, 'remak': 0.042990491362086054, 'road': 0.09219455352279078, 'warry': 0.11447962380297705, 'film': 0.016886078020395573, 'follow': 0.028334528994158857, 'exact': 0.03353579878494298, 'plot': 0.017535131361533313, 'filipino': 0.08480182653135486, 'stryker': 0.08480182653135486, 'wors': 0.03408000706766939, 'far': 0.05100468066697341, 'bad': 0.03307826003200432, 'act': 0.022137203240880893, 'dialog': 0.02755842323696926, 'effect': 0.024828844192497597, 'dub': 0.048035468858358514, 'pac': 0.034873028579013664, 'sequ': 0.0661597525953473, 'list': 0.035252411297038606, 'goe': 0.028042463298464965, 'ita': 0.05777452472487811, 'mad': 0.015156732329265237, 'lit': 0.04104708526962705, 'doz': 0.049014294241580436, 'ripoff': 0.05406448573334878, 'ear': 0.024766712944602098, '80': 0.04650062526497321, 'good': 0.022167814966908405, 'worst': 0.02

In [8]:
#extract unique feature from tfidf_training only

def unique_features(tfidf_values, threshold=0.0):
    features = set()

    # Iterate through each document's TF-IDF values
    for document_tfidf in tfidf_values:
        # Iterate through each feature and its TF-IDF score in the document
        for feature, tfidf_score in document_tfidf.items():
            if tfidf_score > threshold:
                features.add(feature)

    return list(features)

# Set a threshold to filter out features with low TF-IDF scores
threshold_value = 0.1

# Extract unique features from tfidf_training
unique_features_one_training = unique_features(tfidf_one_training, threshold=threshold_value)
unique_features_two_training = unique_features(tfidf_two_training, threshold=threshold_value)
unique_features_three_training = unique_features(tfidf_three_training, threshold=threshold_value)

print("Number of unique features in training set:", len(unique_features_one_training))
print("Sample of unique features:", unique_features_one_training[:20])

print("\nNumber of unique features in training set:", len(unique_features_two_training))
print("Sample of unique features:", unique_features_two_training[:20])

print("\nNumber of unique features in training set:", len(unique_features_three_training))
print("Sample of unique features:", unique_features_three_training[:20])


Number of unique features in training set: 6412
Sample of unique features: ['math', 'lamest', 'unfocus', 'finem', 'quay', 'list', 'serb', 'whoev', 'grinch', 'colem', 'cuthbert', 'ask', 'medus', 'holt', 'postery', 'stitz', 'credit', 'trent', 'golf', 'subvert']

Number of unique features in training set: 5247
Sample of unique features: ['math', 'lamest', 'finem', 'quay', 'serb', 'whoev', 'grinch', 'colem', 'cuthbert', 'ask', 'medus', 'holt', 'postery', 'stitz', 'credit', 'golf', 'subvert', 'sick', 'sil', 'naudet']

Number of unique features in training set: 4518
Sample of unique features: ['carmen', 'caron', 'buy', 'math', 'survival', 'niro', 'lamest', 'briggitta', 'skinhead', 'frying', 'clap-trap', 'bag', 'expletive', 'vincent', 'know.the', 'create', 'excursion', 'funner', 'bear/rat', 'pinter']


In [9]:
#create matrix to pass tfidf values with unique features

def create_tfidf_matrix(tfidf_values, unique_features):
    tfidf_matrix = []

    # Iterate through each document's TF-IDF values
    for document_tfidf in tfidf_values:
        # Create a row for the TF-IDF matrix using unique features
        row = [document_tfidf.get(feature, 0.0) for feature in unique_features]
        # Append the row to the TF-IDF matrix
        tfidf_matrix.append(row)

    return np.array(tfidf_matrix)

# Create TF-IDF matrices for training, development, and testing sets
tfidf_matrix_one_training = create_tfidf_matrix(tfidf_one_training, unique_features_one_training)
tfidf_matrix_one_development = create_tfidf_matrix(tfidf_one_development, unique_features_one_training)
tfidf_matrix_one_testing = create_tfidf_matrix(tfidf_one_testing, unique_features_one_training)

# Create TF-IDF matrices for training, development, and testing sets
tfidf_matrix_two_training = create_tfidf_matrix(tfidf_two_training, unique_features_two_training)
tfidf_matrix_two_development = create_tfidf_matrix(tfidf_two_development, unique_features_two_training)
tfidf_matrix_two_testing = create_tfidf_matrix(tfidf_two_testing, unique_features_two_training)

# Create TF-IDF matrices for training, development, and testing sets
tfidf_matrix_three_training = create_tfidf_matrix(tfidf_three_training, unique_features_three_training)
tfidf_matrix_three_development = create_tfidf_matrix(tfidf_three_development, unique_features_three_training)
tfidf_matrix_three_testing = create_tfidf_matrix(tfidf_three_testing, unique_features_three_training)

print("TF-IDF Matrix for Training Set: \n", tfidf_matrix_one_training)
print("TF-IDF Matrix for Development Set \n:", tfidf_matrix_one_development)
print("TF-IDF Matrix for Testing Set: \n", tfidf_matrix_one_testing)


TF-IDF Matrix for Training Set: 
 [[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.06742591 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.02848982 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
TF-IDF Matrix for Development Set 
: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
TF-IDF Matrix for Testing Set: 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [10]:
# NORMALISATION:
# Frequency normalisation

def normalise_frequency(ngrams):
    for ngram_ in ngrams:
        term_count = Counter(ngram_)
        total_terms = len(ngram_)
        tf_value = {term: count / total_terms for term, count in term_count.items()}
        return tf_value

normalised_one_train = normalise_frequency(ngrams_one_training)
normalised_one_dev = normalise_frequency(ngrams_one_development)
normalised_one_test = normalise_frequency(ngrams_one_testing)

normalised_two_train = normalise_frequency(ngrams_two_training)
normalised_two_dev = normalise_frequency(ngrams_two_development)
normalised_two_test = normalise_frequency(ngrams_two_testing)

normalised_three_train = normalise_frequency(ngrams_three_training)
normalised_three_dev = normalise_frequency(ngrams_three_development)
normalised_three_test = normalise_frequency(ngrams_three_testing)

In [11]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

def naive_bayes_classifier(training_data, training_labels, development_data, development_labels):#testing_data, testing_labels):
    # Initialize the Multinomial Naive Bayes classifier
    nb_classifier = MultinomialNB()

    # Train the classifier on the training data
    nb_classifier.fit(training_data, training_labels)

    # Make predictions on the development data
    predictions_dev = nb_classifier.predict(development_data)
    #test_dev = nb_classifier.predict(testing_data)

    # Evaluate the classifier
    accuracy = accuracy_score(development_labels, predictions_dev)
    #accuracy = accuracy_score(testing_labels, test_dev)
    report = classification_report(development_labels, predictions_dev)

    print("Accuracy:", accuracy)
    print("\nClassification Report:\n", report)

# Example usage:
#naive_bayes_classifier(tfidf_matrix_one_training, train_labels, tfidf_matrix_one_development, dev_labels)
#naive_bayes_classifier(tfidf_matrix_two_training, train_labels, tfidf_matrix_two_development, dev_labels)
#naive_bayes_classifier(tfidf_matrix_three_training, train_labels, tfidf_matrix_three_development, dev_labels)

naive_bayes_classifier(tfidf_matrix_one_training, train_labels, tfidf_matrix_one_testing, test_labels)
naive_bayes_classifier(tfidf_matrix_two_training, train_labels, tfidf_matrix_two_testing, test_labels)
naive_bayes_classifier(tfidf_matrix_three_training, train_labels, tfidf_matrix_three_testing, test_labels)

Accuracy: 0.815

Classification Report:
               precision    recall  f1-score   support

    negative       0.75      0.90      0.82       187
    positive       0.89      0.74      0.81       213

    accuracy                           0.81       400
   macro avg       0.82      0.82      0.81       400
weighted avg       0.83      0.81      0.81       400

Accuracy: 0.8125

Classification Report:
               precision    recall  f1-score   support

    negative       0.75      0.89      0.82       187
    positive       0.89      0.74      0.81       213

    accuracy                           0.81       400
   macro avg       0.82      0.82      0.81       400
weighted avg       0.82      0.81      0.81       400

Accuracy: 0.79

Classification Report:
               precision    recall  f1-score   support

    negative       0.70      0.96      0.81       187
    positive       0.94      0.64      0.77       213

    accuracy                           0.79       400
   ma

In [12]:
import math
from collections import Counter

class NaiveBayesClassifier:
    def __init__(self):
        # Store class probabilities and feature probabilities
        self.priors = {}  # Prior probabilities for each class
        self.likelihoods = {}  # Likelihood probabilities for each feature given each class

    def train(self, X, y):
        # Calculate priors
        total_samples = len(y)
        unique_classes = set(y)

        for class_label in unique_classes:
            # Count occurrences of each class in the training set
            class_count = sum(1 for label in y if label == class_label)
            # Calculate priors
            self.priors[class_label] = class_count / total_samples

        # Calculate likelihoods for each class
        for class_label in unique_classes:
            # Get indices of samples belonging to the current class
            class_indices = [i for i, label in enumerate(y) if label == class_label]
            # Extract data for the current class
            class_data = [X[i] for i in class_indices]
            # Count occurrences of each word in the current class
            class_word_counts = Counter([word for document in class_data for word in document])

            total_words_in_class = sum(class_word_counts.values())

            # Calculate probabilities for each word in the current class
            self.likelihoods[class_label] = {word: count / total_words_in_class for word, count in class_word_counts.items()}

    def predict(self, X):
        predictions = []

        for document in X:
            max_posterior = float('-inf')
            predicted_class = None

            for class_label, prior in self.priors.items():
                posterior = math.log(prior)

                # Calculate the log probability for each word in the document
                for word in document:
                    if word in self.likelihoods[class_label]:
                        posterior += math.log(self.likelihoods[class_label][word])

                # Update the predicted class if the log probability is higher
                if posterior > max_posterior:
                    max_posterior = posterior
                    predicted_class = class_label

            predictions.append(predicted_class)

        return predictions


def train_and_evaluate(X_train, y_train, X_dev, y_dev, feature_set_name):
    # Instantiate the NaiveBayesClassifier
    nb_classifier = NaiveBayesClassifier()

    # Train the classifier
    nb_classifier.train(X_train, y_train)

    # Make predictions on the development set
    dev_predictions = nb_classifier.predict(X_dev)

    # Evaluate the accuracy
    correct_predictions = sum(1 for pred, true_label in zip(dev_predictions, y_dev) if pred == true_label)
    accuracy = correct_predictions / len(y_dev)

    print(f"Accuracy on testing set for {feature_set_name}: {accuracy}")
    report = classification_report(y_dev, dev_predictions)
    print("\nClassification Report:\n", report)

# Example usage on three different feature sets
feature_sets = [
    (feature_selection_one_training, feature_selection_one_development, "Feature Set One"),
    (feature_selection_two_training, feature_selection_two_development, "Feature Set Two"),
    (feature_selection_three_training, feature_selection_three_development, "Feature Set Three")
]

best_accuracy = 0.0
best_feature_set = None


# Example usage on three different feature sets
#train_and_evaluate(feature_selection_one_training, train_labels, feature_selection_one_testing, test_labels, "Feature Set One")
#train_and_evaluate(feature_selection_two_training, train_labels, feature_selection_two_testing, test_labels, "Feature Set Two")
#train_and_evaluate(feature_selection_three_training, train_labels, feature_selection_three_testing, test_labels, "Feature Set Three")

for X_train, X_dev, feature_set_name in feature_sets:
    train_and_evaluate(X_train, train_labels, X_dev, dev_labels, feature_set_name)

    nb_classifier = NaiveBayesClassifier()
    nb_classifier.train(X_train, train_labels)
    dev_predictions = nb_classifier.predict(X_dev)

    correct_predictions = sum(1 for pred, true_label in zip(dev_predictions, dev_labels) if pred == true_label)
    accuracy = correct_predictions / len(dev_labels)

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_feature_set = feature_set_name

print(f"The best feature set is {best_feature_set} with an accuracy of {best_accuracy}")



Accuracy on testing set for Feature Set One: 0.525

Classification Report:
               precision    recall  f1-score   support

    negative       0.50      0.63      0.56       191
    positive       0.56      0.43      0.49       209

    accuracy                           0.53       400
   macro avg       0.53      0.53      0.52       400
weighted avg       0.53      0.53      0.52       400

Accuracy on testing set for Feature Set Two: 0.535

Classification Report:
               precision    recall  f1-score   support

    negative       0.51      0.64      0.57       191
    positive       0.57      0.44      0.49       209

    accuracy                           0.54       400
   macro avg       0.54      0.54      0.53       400
weighted avg       0.54      0.54      0.53       400

Accuracy on testing set for Feature Set Three: 0.515

Classification Report:
               precision    recall  f1-score   support

    negative       0.49      0.63      0.56       191
    pos

In [13]:
# Use Logistic Regression and SVM packages from https://scikit-learn.org/stable/ and evaluate them on each of the three different feature sets you have extracted.
# You are to perform the evaluation on the development set. Pick the best of the three methods you have experimented with

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Function to create a binary matrix for one-hot embeddings
def create_one_hot_matrix(data, unique_features):
    one_hot_matrix = []

    for document in data:
        row = [1 if feature in document else 0 for feature in unique_features]
        one_hot_matrix.append(row)

    return np.array(one_hot_matrix)

def train_and_evaluate_models(unique_features_training_list, feature_selection_training_list,
                               feature_selection_development_list, feature_selection_testing_list,
                               train_labels, dev_labels, test_labels):
    best_accuracy = 0
    best_model = ""
    best_feature_set = None

    # Iterate through each feature set
    for idx, unique_features_training in enumerate(unique_features_training_list):
        feature_selection_training = feature_selection_training_list[idx]
        feature_selection_development = feature_selection_development_list[idx]
        feature_selection_testing = feature_selection_testing_list[idx]

        # Create one-hot matrices
        X_train_one_hot = create_one_hot_matrix(feature_selection_training, unique_features_training)
        X_dev_one_hot = create_one_hot_matrix(feature_selection_development, unique_features_training)
        X_test_one_hot = create_one_hot_matrix(feature_selection_testing, unique_features_training)


        # Logistic Regression
        logreg_model = LogisticRegression()
        logreg_model.fit(X_train_one_hot, train_labels)
        dev_predictions_logreg = logreg_model.predict(X_dev_one_hot)

        # SVM
        svm_model = SVC()
        svm_model.fit(X_train_one_hot, train_labels)
        dev_predictions_svm = svm_model.predict(X_dev_one_hot)

        # Evaluate on development set
        accuracy_logreg = accuracy_score(dev_labels, dev_predictions_logreg)
        print(f"Accuracy on development set (Logistic Regression - Feature Set {idx + 1}): {accuracy_logreg}")

        #report = classification_report(dev_labels, dev_predictions_logreg)
        #print("\nClassification Report:\n", report)

        accuracy_svm = accuracy_score(dev_labels, dev_predictions_svm)
        print(f"Accuracy on development set (SVM - Feature Set {idx + 1}): {accuracy_svm}")

        #report = classification_report(dev_labels, dev_predictions_svm)
        #print("\nClassification Report:\n", report)

        """

        # Logistic Regression
        logreg_model = LogisticRegression()
        logreg_model.fit(X_train_one_hot, train_labels)
        test_predictions_logreg = logreg_model.predict(X_test_one_hot)

        # SVM
        svm_model = SVC()
        svm_model.fit(X_train_one_hot, train_labels)
        test_predictions_svm = svm_model.predict(X_test_one_hot)

        # Evaluate on testing set
        accuracy_logreg = accuracy_score(test_labels, test_predictions_logreg)
        print(f"Accuracy on testing set (Logistic Regression - Feature Set {idx + 1}): {accuracy_logreg}")

        report = classification_report(test_labels, test_predictions_logreg)
        print("\nClassification Report:\n", report)

        accuracy_svm = accuracy_score(test_labels, test_predictions_svm)
        print(f"Accuracy on testing set (SVM - Feature Set {idx + 1}): {accuracy_svm}")

        report = classification_report(test_labels, test_predictions_svm)
        print("\nClassification Report:\n", report)
        """


        # Check if this feature set gives better accuracy
        if accuracy_logreg > best_accuracy:
            best_accuracy = accuracy_logreg
            best_model = "logreg_model"
            best_feature_set = idx + 1

            X_train = X_train_one_hot
            X_dev = X_dev_one_hot
            X_test = X_test_one_hot

        # Check if this feature set gives better accuracy
        if accuracy_svm > best_accuracy:
            best_accuracy = accuracy_svm
            best_model = "svm_model"
            best_feature_set = idx + 1

            X_train = X_train_one_hot
            X_dev = X_dev_one_hot
            X_test = X_test_one_hot

    print(f"\nBest feature set on development set: Feature Set {best_feature_set}")
    print(f"Best model on development set: {best_model}")
    print(f"Accuracy: {best_accuracy}")
    return best_feature_set, best_model, X_train, X_dev, X_test

# Run the function with your feature sets
best_feature_set, best_model, X_train, X_dev, X_test = train_and_evaluate_models(
    [unique_features_one_training, unique_features_two_training, unique_features_three_training],
    [feature_selection_one_training, feature_selection_two_training, feature_selection_three_training],
    [feature_selection_one_development, feature_selection_two_development, feature_selection_three_development],
    [feature_selection_one_testing, feature_selection_two_testing, feature_selection_three_testing],
    train_labels, dev_labels, test_labels
)

#print(best_feature_set, best_model)

Accuracy on development set (Logistic Regression - Feature Set 1): 0.8375
Accuracy on development set (SVM - Feature Set 1): 0.8225
Accuracy on development set (Logistic Regression - Feature Set 2): 0.8025
Accuracy on development set (SVM - Feature Set 2): 0.8375
Accuracy on development set (Logistic Regression - Feature Set 3): 0.785
Accuracy on development set (SVM - Feature Set 3): 0.82

Best feature set on development set: Feature Set 1
Best model on development set: logreg_model
Accuracy: 0.8375


In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Function for hyperparameter optimization for Logistic Regression
def hyperparameter_loreg_optimization(X_train, y_train, X_dev, y_dev, X_test, y_test):
    C_values = [0.1, 1, 10, 100, 1000]
    penalties = ['l2']
    solvers = ['lbfgs', 'liblinear', 'saga']
    max_iters = [500]
    class_weights = ['balanced']

    best_accuracy = 0
    best_hyperparameters = None

    # Iterate through hyperparameter combinations
    for C in C_values:
        for penalty in penalties:
            for solver in solvers:
                for max_iter in max_iters:
                    for class_weight in class_weights:
                        # Create a Logistic Regression model with the current hyperparameters
                        model = LogisticRegression(
                            C=C,
                            penalty=penalty,
                            solver=solver,
                            max_iter=max_iter,
                            class_weight=class_weight
                        )
                        # Train the model on the training set
                        model.fit(X_train, y_train)

                        # Make predictions on the development set
                        dev_predictions = model.predict(X_dev)

                        # Calculate accuracy on the development set
                        accuracy = accuracy_score(y_dev, dev_predictions)

                        # Check if this combination gives better accuracy
                        if accuracy > best_accuracy:
                            best_accuracy = accuracy
                            best_hyperparameters = {
                                'C': C,
                                'penalty': penalty,
                                'solver': solver,
                                'max_iter': max_iter,
                                'class_weight': class_weight
                            }

    print(f"\nBest hyperparameters: {best_hyperparameters}")
    print(f"Best accuracy on development set: {best_accuracy}")

    # Create the best Logistic Regression model using the best hyperparameters
    best_model = LogisticRegression(**best_hyperparameters)

    # Train the best model on the entire training set
    best_model.fit(X_train, y_train)

    # Evaluate on the testing set
    test_predictions = best_model.predict(X_test)
    test_accuracy = accuracy_score(y_test, test_predictions)

    report = classification_report(y_test, test_predictions)
    print("\nClassification Report:\n", report)

    print(f"\nAccuracy on testing set: {test_accuracy}")

# Call the function with your data
hyperparameter_loreg_optimization(X_train, train_labels, X_dev, dev_labels, X_test, test_labels)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



Best hyperparameters: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs', 'max_iter': 500, 'class_weight': 'balanced'}
Best accuracy on development set: 0.8375

Classification Report:
               precision    recall  f1-score   support

    negative       0.83      0.84      0.83       187
    positive       0.86      0.85      0.85       213

    accuracy                           0.84       400
   macro avg       0.84      0.84      0.84       400
weighted avg       0.84      0.84      0.84       400


Accuracy on testing set: 0.8425


In [15]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Function for hyperparameter optimization for SVM
def hyperparameter_svm_optimization(X_train, y_train, X_dev, y_dev, X_test, y_test):
    C_values = [0.1, 1, 10, 100,]
    kernels = ['linear', 'rbf']# 'poly']
    gammas = ['scale']#, 'auto']

    best_accuracy = 0
    best_hyperparameters = None

    # Iterate through hyperparameter combinations
    for C in C_values:
        for kernel in kernels:
            for gamma in gammas:
                # Create an SVM model with the current hyperparameters
                model = SVC(
                    C=C,
                    kernel=kernel,
                    gamma=gamma,
                )
                # Train the model on the training set
                model.fit(X_train, y_train)

                # Make predictions on the development set
                dev_predictions = model.predict(X_dev)

                # Calculate accuracy on the development set
                accuracy = accuracy_score(y_dev, dev_predictions)

                # Check if this combination gives better accuracy
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_hyperparameters = {
                        'C': C,
                        'kernel': kernel,
                        'gamma': gamma,
                    }

    print(f"\nBest hyperparameters: {best_hyperparameters}")
    print(f"Best accuracy on development set: {best_accuracy}")

    # Train the best model on the training set
    best_model = SVC(**best_hyperparameters)
    best_model.fit(X_train, y_train)

    # Evaluate on the testing set
    test_predictions = best_model.predict(X_test)
    test_accuracy = accuracy_score(y_test, test_predictions)
    report = classification_report(y_test, test_predictions)

    print(f"\nAccuracy on testing set: {test_accuracy}")
    print("\nClassification Report:\n", report)

# Call the function with your data
hyperparameter_svm_optimization(X_train, train_labels, X_dev, dev_labels, X_test, test_labels)


Best hyperparameters: {'C': 1, 'kernel': 'rbf', 'gamma': 'scale'}
Best accuracy on development set: 0.8225

Accuracy on testing set: 0.8425

Classification Report:
               precision    recall  f1-score   support

    negative       0.86      0.80      0.83       187
    positive       0.83      0.88      0.86       213

    accuracy                           0.84       400
   macro avg       0.84      0.84      0.84       400
weighted avg       0.84      0.84      0.84       400



In [16]:
# BERT
# This part of your assignment is completely independent of all previous sections.
# Using the same train/dev/test splits as before, use BERT, specifically the base model, to perform the same classification.
# Be sure to experiment with both the cased and uncased versions of the model. You will find the following tutorials helpful in addition to the discussions in the lectures.
# The following tutorials will use the entire IMDB dataset - your assignment requires you to use the custom splits.
# Custom Datasets: https://huggingface.co/transformers/v3.2.0/custom_datasets.html
# IMDB Classification: https://huggingface.co/docs/transformers/tasks/sequence_classification

